In [ ]:
# !pip install selenium webdriver_manager

In [ ]:
# 操作 browser 的 API
from selenium.webdriver.chrome.service import Service
from selenium import webdriver

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 加入行為鍊 ActionChain (在 WebDriver 中模擬滑鼠移動、點繫、拖曳、按右鍵出現選單，以及鍵盤輸入文字、按下鍵盤上的按鈕等)
from selenium.webdriver.common.action_chains import ActionChains

# 強制等待 (執行期間休息一下)
import time 

# 整理 json 使用的工具
import json

# 執行 command 的時候用的
import os

import re

import shutil

from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# Service(executable_path=ChromeDriverManager().install())

driver = webdriver.Chrome(options=options)
# driver = webdriver.Chrome('chromedriver',options=options)

In [ ]:
filePath = "Bookshelf_Selenium"
print("Initializing File")
if os.path.exists(filePath):
    print("Removing Old File...")
    shutil.rmtree(filePath)

    print("Creating New File...")
    os.mkdir(filePath)
else:
    print("Creating New File...")
    os.mkdir(filePath)

Initializing File
Removing Old File...
Creating New File...


In [ ]:
driver.get("https://www.gutenberg.org/browse/languages/zh")

In [ ]:
try:
    titles = WebDriverWait(driver, 5).until(
        EC.presence_of_all_elements_located(
                (By.CSS_SELECTOR, "li.pgdbetext > a")
            )
    )
except TimeoutException:
    print("Books Meun Page 等待逾時!")

In [ ]:
books = [ title.text for title in titles if re.search(r"[\u4E00-\u9FFF]+", title.text) != None]
# books = []
# for title in titles:
#     if re.search(r"[\u4E00-\u9FFF]+", title.text) != None:
#         books.append(title.text)

In [ ]:
len(books)

478

In [ ]:
# 建立行為鍊
ac = ActionChains(driver)

In [ ]:
for name in books:
    try:
        # book_name = WebDriverWait(driver, 5).until(
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located(
                (By.LINK_TEXT, name)
            )
        ).click()
        # ac.click(book_name).perform()
        # book_name.click()
        
    except TimeoutException:
        print("Books Meun Page 等待逾時!")

    try:
        # txt = WebDriverWait(driver, 5).until(
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located(
                (By.LINK_TEXT, "Plain Text UTF-8")
            )
        ).click()
        # ac.click(txt).perform()
        # txt.click()
        
    except TimeoutException:
        print("Select Download Sources Page 等待逾時!")
        
    try:
        texts = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located(
            (By.TAG_NAME, "pre")
            )
        )
        contents = re.findall("[\u4E00-\u9FFF]+[\W]+", texts.text)
        
    except TimeoutException:
        print("Source Page 等待逾時!")
    
    name_ = re.sub(r"\s", " ", name)
    with open(f"{filePath}/{name_}.txt", "a") as f:
        f.write(" ".join(contents))
    
    lst = os.listdir(filePath)
    if len(lst) > 200:
        break
        
    driver.get("https://www.gutenberg.org/browse/languages/zh")